# Import libraries & Data

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
df = pd.read_csv('/content/drive/MyDrive/MSIB Bitlabs Data Analytics for Business/PBL/data_cleaning/transaction.csv', parse_dates=['transaction_created_datetime', 'transaction_updated_datetime'])
# df_hashed = pd.read_csv('/content/drive/MyDrive/MSIB Bitlabs Data Analytics for Business/PBL/data_cleaning/transaction.csv', parse_dates=['transaction_created_datetime', 'transaction_updated_datetime'])
promotion = pd.read_csv('/content/drive/MyDrive/MSIB Bitlabs Data Analytics for Business/PBL/data_cleaning/promotion.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   dpt_id                        50000 non-null  object        
 1   dpt_promotion_id              50000 non-null  object        
 2   buyer_id                      50000 non-null  object        
 3   seller_id                     50000 non-null  object        
 4   transaction_amount            50000 non-null  float64       
 5   payment_method_name           50000 non-null  object        
 6   payment_provider_name         50000 non-null  object        
 7   transaction_created_datetime  50000 non-null  datetime64[ns]
 8   transaction_updated_datetime  50000 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 3.4+ MB


In [18]:
promotion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 4 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   dpt_promotion_id                   728 non-null    object 
 1   promotion_code                     728 non-null    object 
 2   promotion_name                     728 non-null    object 
 3   transaction_promo_cashback_amount  728 non-null    float64
dtypes: float64(1), object(3)
memory usage: 22.9+ KB


# 1.1 Analyze the distribution of transaction amounts

In [19]:
df['transaction_amount'].describe()

,transaction_amount
count,5.000000e+04
mean,1.965125e+07
std,1.057004e+08
min,0.000000e+00
25%,7.000043e+04
50%,1.850172e+06
75%,1.398597e+07
max,2.014010e+10


In [20]:
transaction = df.copy()
# Menghitung jumlah transaksi yang memiliki nilai minimal dan maksimal
min_count = transaction[transaction['transaction_amount']==transaction['transaction_amount'].min()].shape[0]
max_count = transaction[transaction['transaction_amount']==transaction['transaction_amount'].max()].shape[0]
print(f"  - Nilai Minimal Transaksi: {transaction['transaction_amount'].min()} (Banyak transaksi: {min_count} kali)")
print(f"  - Nilai Maksimal Transaksi: {transaction['transaction_amount'].max()} (Banyak transaksi: {max_count} kali)")

  - Nilai Minimal Transaksi: 0.0 (Banyak transaksi: 26 kali)
  - Nilai Maksimal Transaksi: 20140103214.0 (Banyak transaksi: 1 kali)


In [21]:
# Menentukan rentang nilai
bins = [0, 1000, 10**4, 10**5, 10**6, 10**7, 10**8, 10**9, 10**10, 3*(10**10)]
labels = ['0 hingga 1K', '1K hingga 10K', '10K hingga 100K', '100K hingga 1M', '1M hingga 10M',
          '10M hingga 100M', '100M hingga 1B', '1B hingga 10B', '10B hingga 30B']
# Membuat kolom kategori berdasarkan rentang nilai
transaction['amount_range'] = pd.cut(transaction['transaction_amount'], bins=bins, labels=labels, include_lowest=False, right=True)

# Menghitung frekuensi transaksi di setiap rentang
range_counts = transaction['amount_range'].value_counts().sort_index()

# Menampilkan deskripsi statistik
print("Banyaknya Transaction Amount Berdasarkan Rentang Nilai:")
print(f"{bins[0]}: {min_count} transaksi")
for label, count in range_counts.items():
    print(f"Lebih dari {label}: {count} transaksi     {count/len(transaction)*100:.3f}% dari keseluruhan transaksi")


Banyaknya Transaction Amount Berdasarkan Rentang Nilai:
0: 26 transaksi
Lebih dari 0 hingga 1K: 16 transaksi     0.032% dari keseluruhan transaksi
Lebih dari 1K hingga 10K: 538 transaksi     1.076% dari keseluruhan transaksi
Lebih dari 10K hingga 100K: 12454 transaksi     24.908% dari keseluruhan transaksi
Lebih dari 100K hingga 1M: 8726 transaksi     17.452% dari keseluruhan transaksi
Lebih dari 1M hingga 10M: 13380 transaksi     26.760% dari keseluruhan transaksi
Lebih dari 10M hingga 100M: 13228 transaksi     26.456% dari keseluruhan transaksi
Lebih dari 100M hingga 1B: 1628 transaksi     3.256% dari keseluruhan transaksi
Lebih dari 1B hingga 10B: 3 transaksi     0.006% dari keseluruhan transaksi
Lebih dari 10B hingga 30B: 1 transaksi     0.002% dari keseluruhan transaksi


In [23]:
# Membuat histogram dengan Plotly Express
fig = px.histogram(transaction, x='amount_range', title='Distribusi Transaction Amount Berdasarkan Rentang Nilai',
                   category_orders={'amount_range': labels},
                   histfunc='count',
                   labels={'amount_range': 'Rentang Transaction Amount'},
                   text_auto=True)

# Menampilkan plot
fig.show()


# 1.2 Analyze frequencies of buyer-seller pairs

In [24]:
# Analyze frequencies of buyer-seller pairs
buyer_seller_pairs = df.groupby(['buyer_id', 'seller_id']).size().reset_index(name='frequency')
top_buyer_seller_pairs = buyer_seller_pairs.sort_values(by='frequency', ascending=False)
top_buyer_seller_pairs

,buyer_id,seller_id,frequency
470,0bb440f2ae8461ca7b424f9b0efddbb2a1993e07a6d629...,0bb440f2ae8461ca7b424f9b0efddbb2a1993e07a6d629...,1266
698,10f3200ad77826457a7b33726d1ec95de21ee6400745aa...,10f3200ad77826457a7b33726d1ec95de21ee6400745aa...,321
2112,34d1c64bbd54c2912026f19d45b7274b31f3e5f3ca9a81...,34d1c64bbd54c2912026f19d45b7274b31f3e5f3ca9a81...,294
7292,b4c5286fbf6443dd4df37457f3ef23a2b71253b1c86452...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,294
5998,9506dece8982a8d50fbb2924dd510825ea1516c19852aa...,9506dece8982a8d50fbb2924dd510825ea1516c19852aa...,261
...,...,...,...
4785,777506a7051433139e46fb29becf0d74bd32176628b3de...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,1
4779,774bd8a333fd643f1cef296aedaec3981d30431fa1dd7c...,774bd8a333fd643f1cef296aedaec3981d30431fa1dd7c...,1
4775,773172a82ee71d678138ff3d30e9fec84f39975fc017ca...,773172a82ee71d678138ff3d30e9fec84f39975fc017ca...,1
4772,772e85e3fe9e42bb746d6040d0a8ad0daa89eb5488d640...,0ca7a29a6ede6037fc0694c9a4c2f5330459255e8da25a...,1


In [25]:
# Menentukan rentang nilai (thresholds) dan label interval
thresholds = [0, 5, 10, 20, 30, 40, 50, 100, 150, 200, 250, 300, 350, 1250, 1300]
labels = [f"Lebih dari {thresholds[i]} hingga {thresholds[i+1]} kali" for i in range(len(thresholds) - 1)]

# Membuat kolom kategori berdasarkan rentang nilai
buyer_seller_pairs['frequency_range'] = pd.cut(
    buyer_seller_pairs['frequency'],
    bins=thresholds,
    labels=labels,
    include_lowest=False,
    right=True
)

# Menghitung frekuensi transaksi di setiap rentang
range_counts = buyer_seller_pairs['frequency_range'].value_counts().sort_index()

# Menampilkan deskripsi statistik
print("Total Frekuensi Transaksi Setiap Pasangan Buyer-Seller Berdasarkan Rentang Frekuensi:")
for label, count in range_counts.items():
    print(f"{label}: {count} pasangan     ({count/len(buyer_seller_pairs)*100:.2f}% dari total pasangan buyer-seller)")


Total Frekuensi Transaksi Setiap Pasangan Buyer-Seller Berdasarkan Rentang Frekuensi:
Lebih dari 0 hingga 5 kali: 8398 pasangan     (81.11% dari total pasangan buyer-seller)
Lebih dari 5 hingga 10 kali: 1017 pasangan     (9.82% dari total pasangan buyer-seller)
Lebih dari 10 hingga 20 kali: 587 pasangan     (5.67% dari total pasangan buyer-seller)
Lebih dari 20 hingga 30 kali: 158 pasangan     (1.53% dari total pasangan buyer-seller)
Lebih dari 30 hingga 40 kali: 60 pasangan     (0.58% dari total pasangan buyer-seller)
Lebih dari 40 hingga 50 kali: 39 pasangan     (0.38% dari total pasangan buyer-seller)
Lebih dari 50 hingga 100 kali: 63 pasangan     (0.61% dari total pasangan buyer-seller)
Lebih dari 100 hingga 150 kali: 15 pasangan     (0.14% dari total pasangan buyer-seller)
Lebih dari 150 hingga 200 kali: 7 pasangan     (0.07% dari total pasangan buyer-seller)
Lebih dari 200 hingga 250 kali: 5 pasangan     (0.05% dari total pasangan buyer-seller)
Lebih dari 250 hingga 300 kali: 3 p

In [28]:
# Membuat histogram dengan Plotly Express
fig = px.histogram(buyer_seller_pairs, x='frequency_range', title='Total pasangan buyer-seller per frekuensi transaksi berulang',
                   category_orders={'frequency_range': labels},
                   histfunc='count',
                   labels={'frequency_range': 'Frekuensi transaksi berulang'},
                   text_auto=True)

# Menampilkan plot
fig.show()

# 1.3 Analyze the usage of promotions

In [40]:
# Membaca data transaksi dan promosi
transaction_data = df
promotion_data = promotion

# Hanya menyertakan transaksi dengan promosi
promo_transactions = transaction_data[transaction_data["dpt_promotion_id"] != "no promotion"]

In [44]:
promo_transactions.shape[0]

937

In [41]:
print(f"Persentase transaksi menggunakan promo hanya {len(promo_transactions)/len(transaction_data)*100:.2f}%")

Persentase transaksi menggunakan promo hanya 1.87%


In [48]:
# Menampilkan semua promosi beserta jumlah penggunaannya
promo_usage_counts = promo_transactions["dpt_promotion_id"].value_counts()
print(promo_usage_counts.head())

dpt_promotion_id
promotion-219036467    243
promotion-214984720     88
promotion-188676794     69
promotion-674366813     39
promotion-296267691     37
Name: count, dtype: int64


### Distribusi Transaksi Menggunakan Promo

In [42]:
# Kelompokkan berdasarkan buyer_id, lalu hitung jumlah transaksi yang dilakukan setiap buyer
buyer_promo_usage = promo_transactions.groupby('buyer_id').size().reset_index(name='promo_usage')

# Menghitung jumlah buyer berdasarkan banyaknya transaksi promo yang dilakukan
usage_summary = buyer_promo_usage['promo_usage'].value_counts().reset_index()
usage_summary.columns = ['promo_usage', 'buyer_count']

# Menyortir hasil
usage_summary = usage_summary.sort_values(by='promo_usage', ascending=False)

# Menampilkan hasil
print("Jumlah Buyer Berdasarkan Banyaknya Penggunaan Promo:")
for _, row in usage_summary.iterrows():
    print(f"Buyer menggunakan promo sebanyak {row['promo_usage']} kali: {row['buyer_count']}     ({row['buyer_count']/len(buyer_promo_usage)*100:.2f}%)")


Jumlah Buyer Berdasarkan Banyaknya Penggunaan Promo:
Buyer menggunakan promo sebanyak 8 kali: 1     (0.14%)
Buyer menggunakan promo sebanyak 6 kali: 2     (0.27%)
Buyer menggunakan promo sebanyak 5 kali: 2     (0.27%)
Buyer menggunakan promo sebanyak 4 kali: 7     (0.96%)
Buyer menggunakan promo sebanyak 3 kali: 26     (3.56%)
Buyer menggunakan promo sebanyak 2 kali: 109     (14.93%)
Buyer menggunakan promo sebanyak 1 kali: 583     (79.86%)


### Distribusi Transaksi Menggunakan Promo yang Sama

In [43]:
# Kelompokkan berdasarkan buyer_id dan dpt_promotion_id, lalu hitung jumlah transaksi
buyer_promotion_counts = (
    promo_transactions.groupby(['buyer_id', 'dpt_promotion_id'])
    .size()
    .reset_index(name='promo_transaction_count')
)

# Menghitung jumlah pasangan buyer-promo berdasarkan berapa kali kode promo digunakan
usage_summary = buyer_promotion_counts['promo_transaction_count'].value_counts().reset_index()
usage_summary.columns = ['promo_transaction_count', 'pair_count']

# Menyortir hasil
usage_summary = usage_summary.sort_values(by='promo_transaction_count', ascending=False)

# Menampilkan hasil
print("Jumlah Pasangan Buyer-Promo Berdasarkan Banyaknya Penggunaan Promo yang Sama:")
for _, row in usage_summary.iterrows():
    print(f"Buyer menggunakan satu kode promo sebanyak {row['promo_transaction_count']} kali: {row['pair_count']}     ({row['pair_count']/len(buyer_promotion_counts)*100:.2f}%)")


Jumlah Pasangan Buyer-Promo Berdasarkan Banyaknya Penggunaan Promo yang Sama:
Buyer menggunakan satu kode promo sebanyak 5 kali: 2     (0.23%)
Buyer menggunakan satu kode promo sebanyak 4 kali: 1     (0.11%)
Buyer menggunakan satu kode promo sebanyak 3 kali: 4     (0.46%)
Buyer menggunakan satu kode promo sebanyak 2 kali: 44     (5.03%)
Buyer menggunakan satu kode promo sebanyak 1 kali: 823     (94.16%)


#	2. Investigate patterns in repeated transactions or abnormally high transaction values

In [55]:

# Analyze frequencies of buyer-seller pairs
buyer_seller_pairs = df.groupby(['buyer_id', 'seller_id']).size().reset_index(name='frequency')
buyer_seller_pairs["is_self_transaction"] = (buyer_seller_pairs["buyer_id"] == buyer_seller_pairs["seller_id"]).astype(int)
top_buyer_seller_pairs = buyer_seller_pairs.sort_values(by='frequency', ascending=False)
top_buyer_seller_pairs[].head(10)

,buyer_id,seller_id,frequency,is_self_transaction
470,0bb440f2ae8461ca7b424f9b0efddbb2a1993e07a6d629...,0bb440f2ae8461ca7b424f9b0efddbb2a1993e07a6d629...,1266,1
698,10f3200ad77826457a7b33726d1ec95de21ee6400745aa...,10f3200ad77826457a7b33726d1ec95de21ee6400745aa...,321,1
2112,34d1c64bbd54c2912026f19d45b7274b31f3e5f3ca9a81...,34d1c64bbd54c2912026f19d45b7274b31f3e5f3ca9a81...,294,1
7292,b4c5286fbf6443dd4df37457f3ef23a2b71253b1c86452...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,294,0
5998,9506dece8982a8d50fbb2924dd510825ea1516c19852aa...,9506dece8982a8d50fbb2924dd510825ea1516c19852aa...,261,1
1349,2155a0b3ec4ef3cb18f0890eab8177381c7e96dad96297...,2155a0b3ec4ef3cb18f0890eab8177381c7e96dad96297...,248,1
8984,df43724682fbd651e549d3e2043a9d0d2cc4700545863a...,df43724682fbd651e549d3e2043a9d0d2cc4700545863a...,243,1
91,02611e2fdd7d730bddbd654baf24f03a739704bcb34c01...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,216,0
8985,df49a12bd4d8e5cc564d587ddf36c8b15be260b4b048e8...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,215,0
290,0719f2ed66328a141dc1807848bcc72edd027567ae4fc4...,5d2233f5a1a6435891142442fac09a77809d0c16496f07...,210,0


In [56]:
top_buyer_seller_pairs_100 = buyer_seller_pairs[buyer_seller_pairs["frequency"] > 100]
self_transaction_count = top_buyer_seller_pairs_100["is_self_transaction"].sum()
total_top_pairs = len(top_buyer_seller_pairs_100)
self_transaction_percentage = (self_transaction_count / total_top_pairs) * 100

print(f"Total pasangan dengan transaksi > 100: {total_top_pairs}")
print(f"Jumlah self-transaction: {self_transaction_count}")
print(f"Persentase self-transaction: {self_transaction_percentage:.2f}%")


Total pasangan dengan transaksi > 100: 32
Jumlah self-transaction: 24
Persentase self-transaction: 75.00%


### Self Transaction, Transaksi Sangat Besar, Transaksi Berulang

In [ ]:
transaction = df.copy()
# Menghitung jumlah transaksi yang memiliki nilai minimal dan maksimal
min_count = transaction[transaction['transaction_amount']==transaction['transaction_amount'].min()].shape[0]
max_count = transaction[transaction['transaction_amount']==transaction['transaction_amount'].max()].shape[0]
print(f"  - Nilai Minimal Transaksi: {transaction['transaction_amount'].min()} (Banyak transaksi: {min_count} kali)")
print(f"  - Nilai Maksimal Transaksi: {transaction['transaction_amount'].max()} (Banyak transaksi: {max_count} kali)")

  - Nilai Minimal Transaksi: 0.0 (Banyak transaksi: 26 kali)
  - Nilai Maksimal Transaksi: 20140103214.0 (Banyak transaksi: 1 kali)


In [ ]:
# Menentukan rentang nilai
bins = [0, 1000, 10**4, 10**5, 10**6, 10**7, 10**8, 10**9, 10**10, 3*(10**10)]
labels = ['0 hingga 1K', '1K hingga 10K', '10K hingga 100K', '100K hingga 1M', '1M hingga 10M',
          '10M hingga 100M', '100M hingga 1B', '1B hingga 10B', '10B hingga 30B']
# Membuat kolom kategori berdasarkan rentang nilai
transaction['amount_range'] = pd.cut(transaction['transaction_amount'], bins=bins, labels=labels, include_lowest=False, right=True)

# Menghitung frekuensi transaksi di setiap rentang
range_counts = transaction['amount_range'].value_counts().sort_index()

# Menghitung self-transactions untuk setiap rentang
self_transactions = (
    transaction[transaction['buyer_id'] == transaction['seller_id']]
    .groupby('amount_range', observed=False)  # Tambahkan observed=False
    .size()
    .reindex(labels, fill_value=0)  # Pastikan rentang yang kosong tetap dihitung
)

# Menampilkan hasil sesuai format
print("Frekuensi Transaksi dengan Diri Sendiri Berdasarkan Rentang Nilai Transaksi:")

# Filter transaksi dengan transaction_amount == 0
transaction_zero = transaction[transaction['transaction_amount'] == 0]
self_count = transaction_zero[transaction_zero['buyer_id'] == transaction_zero['seller_id']].shape[0]
percentage = (self_count / transaction_zero.shape[0] * 100)
print(f"0: {transaction_zero.shape[0]} transaksi, {self_count} transaksi merupakan self-transaction, ({percentage:.2f}%) dari keseluruhan transaksi di rentang ini.")

for label, total_count in range_counts.items():
    self_count = self_transactions[label] if label in self_transactions.index else 0
    percentage = (self_count / total_count * 100) if total_count > 0 else 0
    print(f"Lebih dari {label}: {total_count} transaksi, {self_count} transaksi merupakan self-transaction, ({percentage:.2f}%) dari keseluruhan transaksi di rentang ini.")


Frekuensi Transaksi dengan Diri Sendiri Berdasarkan Rentang Nilai Transaksi:
0: 26 transaksi, 1 transaksi merupakan self-transaction, (3.85%) dari keseluruhan transaksi di rentang ini.
Lebih dari 0 hingga 1K: 16 transaksi, 8 transaksi merupakan self-transaction, (50.00%) dari keseluruhan transaksi di rentang ini.
Lebih dari 1K hingga 10K: 538 transaksi, 500 transaksi merupakan self-transaction, (92.94%) dari keseluruhan transaksi di rentang ini.
Lebih dari 10K hingga 100K: 12454 transaksi, 6643 transaksi merupakan self-transaction, (53.34%) dari keseluruhan transaksi di rentang ini.
Lebih dari 100K hingga 1M: 8726 transaksi, 5749 transaksi merupakan self-transaction, (65.88%) dari keseluruhan transaksi di rentang ini.
Lebih dari 1M hingga 10M: 13380 transaksi, 6874 transaksi merupakan self-transaction, (51.38%) dari keseluruhan transaksi di rentang ini.
Lebih dari 10M hingga 100M: 13228 transaksi, 3364 transaksi merupakan self-transaction, (25.43%) dari keseluruhan transaksi di rentang